In [7]:

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import openpyxl
import numpy as np
from api_config import googleAPI

In [40]:
data = pd.read_excel(r"C:\restaurant-form\backend\templates\Ontact_lunch.xlsx")

In [41]:
data['주소'] = 0

In [42]:
lst = data['가게이름']
lst

0             연어초밥
1               불밥
2               빗갈
3                삭
4              김가네
5             단풍나무
6                유
7             카츠난반
8           사장님돈까스
9               가미
10        롯데리아 이대점
11        서브웨이 이대점
12             우리집
13             박가네
14            까이식당
15         고냉지김치찌개
16             훈카츠
17             청화원
18           한와담소반
19            정육면체
20            가야라멘
21             미분당
22            카츠난반
23            한끼마끼
24             박가네
25             우리집
26      돼지궁둥이소풍가는날
27           오사카우동
28             일등집
29             유소바
30             모미지
31           타이스토리
32    방콕익스프레스 신촌본점
33             포가레
34          천궁전마라탕
Name: 가게이름, dtype: object

In [43]:
data.head()

,가게이름,주메뉴(3개까지),카테고리,가격대(24년기준),주소
0,연어초밥,"초밥, 알밥",일식,11000~15000원,0
1,불밥,"(돼지고기,오징어,닭) 볶음 1인당 선택가능",한식,10000원,0
2,빗갈,육회비빔밥,한식,6000원~8000원,0
3,삭,"튀김, 떡볶이, 순대, 어묵",분식,NaN,0
4,김가네,"김밥, 라면, 우동 등",분식,5000원 ~ 10000원,0


In [44]:
# 웹 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # 브라우저를 열지 않고 실행
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# ChromeDriver 경로 설정
driver = webdriver.Chrome(options=options)

# 결과를 저장할 리스트
results = []

# 카카오 지도 접속
driver.get('https://map.kakao.com/')


for shop_name in lst:
    try:
        # 검색어 입력
        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#search\\.keyword\\.query'))
        )
        search_box.clear()  # 입력란 초기화
        search_box.send_keys(shop_name)
        search_box.send_keys(Keys.RETURN)
        
        # 검색 결과 로딩 대기 및 도로명 주소 추출
        road_address_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#info\\.search\\.place\\.list > li.PlaceItem > div.info_item > div.addr > p:nth-child(1)'))
        )
        road_address = road_address_element.text
        print(f'가게 이름: {shop_name}, 도로명 주소: {road_address}')
        
        # 결과 저장
        data.loc[data['가게이름']==shop_name, '주소'] = road_address
        
        # 입력란 지우기
        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#search\\.keyword\\.query'))
        )
        for _ in range(len(shop_name)):
            search_box.send_keys(Keys.BACKSPACE)
        time.sleep(1)  # 잠시 대기
        
    except Exception as e:
        print(f'{shop_name}의 주소를 가져오는 중 오류 발생: {e}')

driver.quit()

print(data)

가게 이름: 연어초밥, 도로명 주소: 서울 서대문구 이화여대1길 42-1
가게 이름: 불밥, 도로명 주소: 서울 서대문구 이화여대1길 42-1
가게 이름: 빗갈, 도로명 주소: 서울 서대문구 이화여대8길 11 2층
가게 이름: 삭, 도로명 주소: 서울 서대문구 명물길 37
가게 이름: 김가네, 도로명 주소: 서울 서대문구 이화여대3길 12 1층
가게 이름: 단풍나무, 도로명 주소: 서울 서대문구 이화여대7길 30
가게 이름: 유, 도로명 주소: 서울 마포구 도화4길 73 도화현대홈타운 상가 지하1층
가게 이름: 카츠난반, 도로명 주소: 서울 마포구 독막로 67-11
가게 이름: 사장님돈까스, 도로명 주소: 서울 서대문구 이화여대7길 37 1층
가게 이름: 가미, 도로명 주소: 서울 서대문구 이화여대7길 11 1층
가게 이름: 롯데리아 이대점, 도로명 주소: 서울 서대문구 이화여대8길 2 301-나,302호
가게 이름: 서브웨이 이대점, 도로명 주소: 서울 서대문구 이화여대길 59 1층
가게 이름: 우리집, 도로명 주소: 서울 서대문구 이화여대길 34 1층
가게 이름: 박가네, 도로명 주소: 서울 서대문구 이화여대길 72-6 1층
가게 이름: 까이식당, 도로명 주소: 서울 서대문구 이화여대3길 12-4 1층
가게 이름: 고냉지김치찌개, 도로명 주소: 서울 서대문구 이화여대2가길 24 1층
가게 이름: 훈카츠, 도로명 주소: 서울 서대문구 연세로2마길 30
가게 이름: 청화원, 도로명 주소: 서울 서대문구 이화여대1길 42-14 1층
가게 이름: 한와담소반, 도로명 주소: 서울 서대문구 명물길 10 1층
가게 이름: 정육면체, 도로명 주소: 서울 서대문구 신촌로 83 현대백화점 10층
가게 이름: 가야라멘, 도로명 주소: 서울 서대문구 연세로5다길 22-8 1층
가게 이름: 미분당, 도로명 주소: 부산 부산진구 중앙대로691번가길 15
가게 이름: 카츠난반, 도로명 주소: 서울 서대문구 연세로5길 26-7
가게 이름: 한끼마끼, 도로명 주소: 서울 서대문구 이화여

In [45]:
data

,가게이름,주메뉴(3개까지),카테고리,가격대(24년기준),주소
0,연어초밥,"초밥, 알밥",일식,11000~15000원,서울 서대문구 이화여대1길 42-1
1,불밥,"(돼지고기,오징어,닭) 볶음 1인당 선택가능",한식,10000원,서울 서대문구 이화여대1길 42-1
2,빗갈,육회비빔밥,한식,6000원~8000원,서울 서대문구 이화여대8길 11 2층
3,삭,"튀김, 떡볶이, 순대, 어묵",분식,NaN,서울 서대문구 명물길 37
4,김가네,"김밥, 라면, 우동 등",분식,5000원 ~ 10000원,서울 서대문구 이화여대3길 12 1층
5,단풍나무,"뼈칼국수, 비빔칼국수",분식,8000원~1000원,서울 서대문구 이화여대7길 30
6,유,"간짜장, 삼선짬뽕, 덮밥",중식,8000원 ~ 15000원,서울 마포구 도화4길 73 도화현대홈타운 상가 지하1층
7,카츠난반,"돈가스,덮밥,치킨난반",한식,9000원 ~ 14000원,서울 서대문구 연세로5길 26-7
8,사장님돈까스,"돈까스,메밀",한식,11000원~15000원,서울 서대문구 이화여대7길 37 1층
9,가미,"닭칼국수(매운맛), 제육덮밥, 가미우동",한식,10000원~15000원,서울 서대문구 이화여대7길 11 1층


In [8]:
import requests
from math import radians, cos, sin, asin, sqrt

API_KEY = googleAPI
BASE_URL = 'https://maps.googleapis.com/maps/api/geocode/json'

# 회사의 위도와 경도
company_lat = 37.559518
company_lon = 126.9479577

# 주소를 위도와 경도로 변환하는 함수
def get_coordinates(address):
    url = 'https://maps.googleapis.com/maps/api/geocode/json'
    params = {
        'address': address,
        'key': API_KEY
    }
    response = requests.get(url, params=params)
    results = response.json().get('results')
    if results:
        location = results[0].get('geometry').get('location')
        return location.get('lat'), location.get('lng')
    return None, None

# Haversine 공식을 사용하여 두 좌표 사이의 거리를 계산하는 함수
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # 지구 반지름 (km)
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a))
    distance_km = R * c
    distance_m = int(distance_km * 1000)  # 미터 단위로 변환하고 정수로 변환
    return distance_m


In [47]:
# 각 주소에 대해 위도와 경도를 구하고 데이터프레임에 추가
data['위도'], data['경도'] = zip(*data['주소'].apply(get_coordinates))

# 회사와의 거리를 계산하여 추가
data['거리'] = data.apply(lambda row: haversine(company_lat, company_lon, row['위도'], row['경도']) if pd.notnull(row['위도']) and pd.notnull(row['경도']) else None, axis=1)

In [54]:
# 삭제할 가게 이름 리스트
delete_list = ['유', '방콕익스프레스 신촌본점']

# 특정 값이 포함된 행 삭제
data = data[~data['가게이름'].isin(delete_list)].reset_index(drop=True)

In [56]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   가게이름        33 non-null     object 
 1   주메뉴(3개까지)   32 non-null     object 
 2   카테고리        33 non-null     object 
 3   가격대(24년기준)  12 non-null     object 
 4   주소          33 non-null     object 
 5   위도          33 non-null     float64
 6   경도          33 non-null     float64
 7   거리          33 non-null     int64  
dtypes: float64(2), int64(1), object(5)
memory usage: 2.2+ KB


In [59]:
import pandas as pd
from sqlalchemy import create_engine

In [61]:
engine = create_engine('sqlite:///restaurants.db')

# 데이터베이스에 데이터프레임 삽입
data.to_sql('restaurants', con=engine, if_exists='replace', index=False)

33

In [63]:
import json

# 데이터프레임을 JSON 형식으로 변환
def dataframe_to_json(df):
    # 필요한 컬럼만 추출하고 이름을 변경합니다.
    df = df[['가게이름', '주메뉴(3개까지)', '카테고리', '가격대(24년기준)', '위도', '경도', '거리']]
    df.columns = ['name', 'menu', 'category', 'price_range', 'latitude', 'longitude', 'distance']

    # 거리(m)를 문자열 형식으로 변환
    df['distance'] = df['distance'].astype(str) + 'm'

    # JSON으로 변환
    json_data = {'restaurants': df.to_dict(orient='records')}
    return json_data

# 데이터프레임을 JSON으로 변환
json_data = dataframe_to_json(data)

# JSON 파일로 저장
with open('restaurants.json', 'w', encoding='utf-8') as f:
    json.dump(json_data, f, ensure_ascii=False, indent=4)

# 결과 출력 (JSON 형식으로 변환된 데이터)
print(json.dumps(json_data, ensure_ascii=False, indent=4))

{
    "restaurants": [
        {
            "name": "연어초밥",
            "menu": "초밥, 알밥",
            "category": "일식",
            "price_range": "11000~15000원",
            "latitude": 37.5572679,
            "longitude": 126.9433882,
            "distance": "474m"
        },
        {
            "name": "불밥",
            "menu": "(돼지고기,오징어,닭) 볶음 1인당 선택가능",
            "category": "한식",
            "price_range": "10000원",
            "latitude": 37.5572679,
            "longitude": 126.9433882,
            "distance": "474m"
        },
        {
            "name": "빗갈",
            "menu": "육회비빔밥",
            "category": "한식",
            "price_range": "6000원~8000원",
            "latitude": 37.5588434,
            "longitude": 126.9464048,
            "distance": "156m"
        },
        {
            "name": "삭",
            "menu": "튀김, 떡볶이, 순대, 어묵",
            "category": "분식",
            "price_range": NaN,
            "latitude": 37.55856240000001,
            "longitud

C:\Users\a\AppData\Local\Temp\ipykernel_8428\1916650302.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['distance'] = df['distance'].astype(str) + 'm'
